#INGEST STATEMENT

In [1]:
import import_ipynb

In [2]:
from modules.ingest_statement import *

importing Jupyter notebook from C:\Users\nomnc\OneDrive\Fin_dump\fin_mgmt\modules\ingest_statement.ipynb


In [3]:
X=create_onedrive_directdownload('https://1drv.ms/u/s!AiVkYGtq2xelg8Qxc1789sNVsxdGXw?e=cuKi3I')

## Read Starling File

In [4]:
import pandas as pd
pdf = pd.read_csv(X)

In [5]:
star_tbl=spark.createDataFrame(pdf)

## Convert columns to lower case

In [7]:
for col in star_tbl.columns:
  star_tbl = star_tbl.withColumnRenamed(col, col.lower())

## Rename 'amount,balance,counter party,reference and spending category' attributes

In [8]:
star_tbl=star_tbl.withColumnRenamed('amount (gbp)','amount')

In [9]:
star_tbl=star_tbl.withColumnRenamed('balance (gbp)','balance')

In [10]:
star_tbl=star_tbl.withColumnRenamed('spending category','spending_category')

In [11]:
star_tbl=star_tbl.withColumnRenamed('counter party','merchant')

In [12]:
star_tbl=star_tbl.withColumnRenamed('reference','description')

## Drop 'notes and balance' attributes

In [13]:
star_tbl=star_tbl.drop('notes')

In [14]:
star_tbl=star_tbl.drop('balance')

In [15]:
star_tbl.show()

+----------+-------------------+--------------------+--------------------+-------+------------------+
|      date|           merchant|         description|                type| amount| spending_category|
+----------+-------------------+--------------------+--------------------+-------+------------------+
|06/06/2022|      Nomzamo Ncube|            transfer|      FASTER PAYMENT|    5.0|            INCOME|
|07/06/2022|              Vimeo|VMO*VIMEO.COM    ...|      ONLINE PAYMENT|  -3.56|          SHOPPING|
|19/06/2022|      Nomzamo Ncube|            transfer|      FASTER PAYMENT|    5.0|            INCOME|
|20/06/2022|              Vimeo|VMO*VIMEO.COM    ...|      ONLINE PAYMENT|  -3.64|          SHOPPING|
|04/07/2022|             iTunes|APPLE.COM/BILL   ...|      ONLINE PAYMENT|  -0.79|     ENTERTAINMENT|
|05/07/2022|          HMRC PAYE|   TSG88093689685438|       DIRECT CREDIT| 1246.6|          PAYMENTS|
|05/07/2022|             iTunes|APPLE.COM/BILL   ...|      ONLINE PAYMENT|  -2.99|

In [16]:
from pyspark.sql.functions import when,lit

## Create 'money_in' attribute

In [17]:
star_tbl=star_tbl.withColumn("money_in", \
              when((star_tbl.amount > 0),lit(star_tbl.amount)) \
              .otherwise(lit(''))\
             )

In [18]:
#star_tbl.select('date','amount','conversionrate','money_in').filter("conversionrate=='0'").show()

## Create 'money_out' attribute

In [34]:
star_tbl=star_tbl.withColumn("money_out", \
              when((star_tbl.amount < 0),lit(star_tbl.amount*-1)) \
              .otherwise(lit(''))\
             )

AttributeError: 'DataFrame' object has no attribute 'amount'

In [21]:
star_tbl.printSchema()

root
 |-- date: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- description: string (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- spending_category: string (nullable = true)
 |-- money_in: string (nullable = true)
 |-- money_out: string (nullable = true)



## Drop 'amount' attribute

In [22]:
star_tbl=star_tbl.drop('amount')

In [23]:
star_tbl.printSchema()

root
 |-- date: string (nullable = true)
 |-- merchant: string (nullable = true)
 |-- description: string (nullable = true)
 |-- type: string (nullable = true)
 |-- spending_category: string (nullable = true)
 |-- money_in: string (nullable = true)
 |-- money_out: string (nullable = true)



In [24]:
from pyspark.sql import functions as F

In [27]:
star_tbl=star_tbl.withColumn('date', F.col('date').cast("timestamp"))

In [28]:
star_tbl.show()

+----+-------------------+--------------------+--------------------+------------------+--------+---------+
|date|           merchant|         description|                type| spending_category|money_in|money_out|
+----+-------------------+--------------------+--------------------+------------------+--------+---------+
|null|      Nomzamo Ncube|            transfer|      FASTER PAYMENT|            INCOME|     5.0|         |
|null|              Vimeo|VMO*VIMEO.COM    ...|      ONLINE PAYMENT|          SHOPPING|        |    -3.56|
|null|      Nomzamo Ncube|            transfer|      FASTER PAYMENT|            INCOME|     5.0|         |
|null|              Vimeo|VMO*VIMEO.COM    ...|      ONLINE PAYMENT|          SHOPPING|        |    -3.64|
|null|             iTunes|APPLE.COM/BILL   ...|      ONLINE PAYMENT|     ENTERTAINMENT|        |    -0.79|
|null|          HMRC PAYE|   TSG88093689685438|       DIRECT CREDIT|          PAYMENTS|  1246.6|         |
|null|             iTunes|APPLE.COM/B

## add buffer attributes to prepare for union

In [30]:
star_tbl=star_tbl.select(
"date",
"merchant",
"description",
"money_out",
"money_in",
lit('starling').alias("account"),
lit('UK').alias("account_country"),
"spending_category",
"type",
lit('').alias("original_currency_amount"),
lit('GBP').alias("currency"),
lit('').alias("conversion_rate"),
lit('').alias("merchant_town"),
lit('').alias("merchant_postcode"),
lit('').alias("merchant_country")
)

In [31]:
star_tbl.show()

+----+-------------------+--------------------+---------+--------+--------+---------------+------------------+--------------------+------------------------+--------+---------------+-------------+-----------------+----------------+
|date|           merchant|         description|money_out|money_in| account|account_country| spending_category|                type|original_currency_amount|currency|conversion_rate|merchant_town|merchant_postcode|merchant_country|
+----+-------------------+--------------------+---------+--------+--------+---------------+------------------+--------------------+------------------------+--------+---------------+-------------+-----------------+----------------+
|null|      Nomzamo Ncube|            transfer|         |     5.0|starling|             UK|            INCOME|      FASTER PAYMENT|                        |     GBP|               |             |                 |                |
|null|              Vimeo|VMO*VIMEO.COM    ...|    -3.56|        |starling| 

## Correct Data types to prepare for Union

In [32]:
star_tbl.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- merchant: string (nullable = true)
 |-- description: string (nullable = true)
 |-- money_out: string (nullable = true)
 |-- money_in: string (nullable = true)
 |-- account: string (nullable = false)
 |-- account_country: string (nullable = false)
 |-- spending_category: string (nullable = true)
 |-- type: string (nullable = true)
 |-- original_currency_amount: string (nullable = false)
 |-- currency: string (nullable = false)
 |-- conversion_rate: string (nullable = false)
 |-- merchant_town: string (nullable = false)
 |-- merchant_postcode: string (nullable = false)
 |-- merchant_country: string (nullable = false)

